In [1]:
import pandas as pd

X = pd.read_csv('lyric_data.csv')

In [2]:
X.keys()

Index(['Unnamed: 0', 'file', 'artist', 'song', 'lyrics', 'tags', 'timestamp',
       'id'],
      dtype='object')

In [3]:

#for i in range(10):
#    print(X.loc[i,'tags'] , X.loc[i,'lyrics'])

In [2]:
import pandas as pd
import numpy as np
from utils import *
import pickle 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import hamming_loss
import time

n = 20000

num_tags = 1000

#load all lyric data into pandas dataframe
df = pd.read_csv('lyric_data.csv', index_col=0)

# Sometimes the API returns an error message rather than actual lyrics. This removes it
bad_song = df['lyrics'].value_counts().index[0]
df[df['lyrics'] == bad_song] = ''

# only take the ones that we have data for
df.fillna('', inplace=True)
df = df[df['lyrics'] != '']
df = df.drop_duplicates(['artist','song'])

# List of list of tags for each example
tags = [clean_tags(raw) for raw in list(df['tags'])]

# list of tuples of (tag, frequency) in desending order
tf = tag_freq(tags)

# Choose which tags to restrict model too
important_tags = [x[0] for x in tf[0:num_tags]]
important_tags = dict(zip(important_tags, range(len(important_tags))))

In [4]:
tf[0:100]

[('rock', 52836),
 ('pop', 32077),
 ('alternative', 31191),
 ('indie', 25683),
 ('favorites', 20974),
 ('female vocalists', 19570),
 ('alternative rock', 18508),
 ('Love', 17287),
 ('metal', 16222),
 ('00s', 15679),
 ('singer-songwriter', 15342),
 ('beautiful', 14402),
 ('indie rock', 14327),
 ('Awesome', 13566),
 ('classic rock', 13555),
 ('male vocalists', 13310),
 ('Mellow', 12727),
 ('american', 12482),
 ('90s', 12462),
 ('electronic', 12103),
 ('hard rock', 12079),
 ('punk', 11882),
 ('british', 11866),
 ('80s', 11842),
 ('seen live', 11233),
 ('folk', 11217),
 ('dance', 10580),
 ('soul', 9908),
 ('chill', 9535),
 ('acoustic', 9404),
 ('Favorite', 9393),
 ('chillout', 9391),
 ('Favourites', 9112),
 ('female vocalist', 8702),
 ('heavy metal', 8338),
 ('cool', 8333),
 ('oldies', 8082),
 ('country', 8006),
 ('blues', 7624),
 ('70s', 7312),
 ('punk rock', 7244),
 ('guitar', 7155),
 ('catchy', 7077),
 ('indie pop', 7043),
 ('sad', 6815),
 ('sexy', 6756),
 ('favorite songs', 6656),
 ('e

In [17]:

def load_glove(vocab):
    embedding_mat = []
    new_vocab = {}

    count = 0
    with open('glove.6B.300d.txt') as f:

        for i, line in enumerate(f):

            s = line.split()

            if s[0] in vocab:
                embedding_mat.append(np.asarray(s[1:]))
                new_vocab[s[0]] = count
                count += 1

                if len(list(new_vocab.keys())) == len(list(vocab.keys())):
                    return new_vocab, np.array(embedding_mat)

    #embedding_mat.append(np.random.randn(50))

    return new_vocab, np.array(embedding_mat).astype(np.float32())

vocab = dict(tf[0:1000])
found_tags , glove = load_glove(vocab)


In [18]:
glove.shape

(426, 300)

In [20]:
from sklearn.metrics.pairwise import pairwise_distances

def nearest_neighbors(X,X_test=None):

    if X_test is not None:
        D = pairwise_distances(X_test,X)
    else:
        D = pairwise_distances(X)

    neighbors = D.argpartition( 3 , axis=1)[:,:5]
    
    return(neighbors,D)

inv_map = {v: k for k, v in found_tags.items()}

N,D = nearest_neighbors(glove)

for tag,i in found_tags.items():
    
    n = N[i]
    d = D[i]
    print(tag, [ inv_map[x] for x in n] ,[ d[x] for x in n] )
#tag = 'hardcore'
#i = 279

#print(nearest_neighbors(glove)[i])

collection ['unique', 'collection', 'work', 'poetry', 'covers'] [6.5750508, 0.0, 6.6440916, 6.9400468, 7.0624237]
underground ['underground', 'garage', '1970s', '1980s', 'places'] [0.0, 6.8612852, 7.2582626, 7.310554, 7.5712223]
5 ['5', '4', '3', '2', '1'] [0.0, 1.7553357, 2.1903257, 2.9842746, 3.7163093]
instrumental ['instrumental', 'vocal', 'music', 'inspiring', 'acoustic'] [0.0, 5.6878576, 6.3050756, 6.3291268, 6.3417892]
songwriter ['singer', 'singer-songwriter', 'songwriter', 'pop', 'music'] [4.5199947, 5.0021033, 0.0, 6.5886998, 6.7882776]
punk ['punk', 'hardcore', 'indie', 'rock', 'goth'] [0.0, 5.2834358, 5.9053178, 5.9814095, 6.1289954]
brutal ['brutal', 'intense', '1990s', '1980s', 'dramatic'] [0.0, 7.0312061, 7.0932741, 7.249577, 7.2981911]
1994 ['1994', '1993', '1995', '1992', '1996'] [0.0, 2.0043573, 2.2152002, 2.4929924, 2.5679531]
eddie ['eddie', 'crazy', 'singer', 'songwriter', 'best'] [0.0, 7.2117009, 7.358429, 7.4344049, 7.4507785]
electro ['electro', 'techno', 'elect

In [41]:
sum([x[1] for x in low_freq_tags])

265276

In [14]:
inv_map = {v: k for k, v in found_tags.items()}
inv_map

{0: 'a',
 1: 'new',
 2: 'other',
 3: 'world',
 4: 'us',
 5: 'country',
 6: 'american',
 7: 'work',
 8: 'party',
 9: 'old',
 10: '1',
 11: 'political',
 12: 'life',
 13: 'good',
 14: 'top',
 15: '2',
 16: 'sunday',
 17: 'best',
 18: 'power',
 19: 'me',
 20: 'european',
 21: 'british',
 22: '3',
 23: 'death',
 24: 'french',
 25: 'great',
 26: 'night',
 27: 'music',
 28: '4',
 29: '5',
 30: 'club',
 31: 'america',
 32: 'college',
 33: 'germany',
 34: 'german',
 35: 'london',
 36: 'special',
 37: 'england',
 38: 'real',
 39: '2008',
 40: '2006',
 41: 'australia',
 42: 'energy',
 43: '2007',
 44: '2010',
 45: '2000',
 46: 'live',
 47: '2009',
 48: '2004',
 49: '2005',
 50: 'english',
 51: 'test',
 52: 'summer',
 53: 'texas',
 54: 'morning',
 55: 'canada',
 56: '2001',
 57: '2003',
 58: 'australian',
 59: '2002',
 60: 'popular',
 61: '1998',
 62: 'light',
 63: '1996',
 64: '1999',
 65: 'beat',
 66: '1994',
 67: 'buy',
 68: '1997',
 69: '1995',
 70: 'italian',
 71: 'heart',
 72: 'heavy',
 73:

In [24]:
df.shape

(187796, 7)

In [43]:
mask = df['tags'].apply(lambda x: 'master-musician' in x)

In [44]:
df.keys()

Index(['file', 'artist', 'song', 'lyrics', 'tags', 'timestamp', 'id'], dtype='object')

In [47]:
df['artist'][mask]

8728           Stevie Wonder
12722      Buckshot LeFonque
15988              B.B. King
35575               Kid Rock
55811           Quincy Jones
88762     Harry Connick, Jr.
99135           Jimi Hendrix
101687    Harry Connick, Jr.
128683          Jimi Hendrix
134606             Sheila E.
145402    Harry Connick, Jr.
189493         Stevie Wonder
193620         Stevie Wonder
210906    Harry Connick, Jr.
246860            John Mayer
280590         Lenny Kravitz
350589          Eric Clapton
351720    Harry Connick, Jr.
362735          Jimi Hendrix
412574         Stevie Wonder
Name: artist, dtype: object

In [34]:
t = df.drop_duplicates(['artist','song'])

In [35]:
t.shape

(159641, 7)

In [48]:
df[df['artist'] =='Jimi Hendrix'].shape

(59, 7)

In [49]:
song_counts = df['artist'].value_counts()

In [54]:
len(song_counts)

12847

In [56]:
(song_counts ==2).sum()

1219

In [12]:

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

#torch.Size([64, 49, 256])


X = torch.randn(64,256,50)

u = torch.randn(64,1,256)

torch.bmm(u,X).size()

torch.Size([64, 1, 50])

In [13]:
a = torch.randn(1,1,256)

In [16]:
a.expand(64,1,256).size()

torch.Size([64, 1, 256])

In [1]:
import pandas as pd
import pickle
import numpy as np
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
import pickle
import torchvision
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
import time
import sys
from utils import *
from networks import *
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import hamming_loss

n = 5000
seq_len = 20
h = 512
num_tags = 30
batch_size = 64

gpu = False

df = pd.read_csv('lyric_data.csv', index_col=0)#.iloc[0:200000]
#df = pd.read_csv('lyric_data_small.csv', index_col=0)


# Sometimes the API returns an error message rather than actual lyrics. This removes it
bad_song = df['lyrics'].value_counts().index[0]
df[df['lyrics'] == bad_song] = ''

# only take the ones that we have data for
df.fillna('', inplace=True)
df = df[df['lyrics'] != '']

# List of list of tags for each example
tags = [clean_tags(raw) for raw in list(df['tags'])]

# list of tuples of (tag, frequency) in desending order
tf = tag_freq(tags)

# Choose which tags to restrict model too
important_tags = [x[0] for x in tf[0:num_tags]]
important_tags = dict(zip(important_tags, range(len(important_tags))))

# maps each of the tags int 'tags' to an int index
indices = tag2index(tags, important_tags)

# Convert indices to binary vectors of tags
y = np.zeros((len(indices), num_tags))
for i, tags in enumerate(indices):
    for tag in tags:
        y[i, tag] = 1

# Build vocabulary and tokenizer
vect = CountVectorizer(max_features=n, stop_words='english')
vect.fit(df['lyrics'])
vocab = vect.vocabulary_
tok = vect.build_analyzer()



torch.Size([64, 100])


FileNotFoundError: [Errno 2] No such file or directory: '/home/zz1409/glove.6B.200d.txt'

In [5]:
def load_glove(vocab):
    embedding_mat = [np.zeros(50)]
    new_vocab = {}

    count = 0

    with open('/Users/Hadoop/Desktop/glove.6B.50d.txt') as f:

        for i, line in enumerate(f):

            s = line.split()

            if s[0] in vocab:
                embedding_mat.append(np.asarray(s[1:]))
                new_vocab[s[0]] = count
                count += 1

                if len(list(new_vocab.keys())) == len(list(vocab.keys())):
                    return new_vocab, np.array(embedding_mat)

    embedding_mat.append(np.random.randn(50))
                         
    return new_vocab, np.array(embedding_mat).astype(np.float32())

# Load glove vectors for word embedding
vocab, glove = load_glove(vocab)


# Convert text to sequence input
features = df['lyrics'].apply(lambda x: sent2seq(x, vocab, tok, seq_len))
features = np.array(list(features))

features = torch.from_numpy(features)
y = torch.from_numpy(y).float()
glove = torch.from_numpy(glove)


train_idx = int(np.floor(features.size()[0] * 8 / 10))

train_loader = torch.utils.data.TensorDataset(features[:train_idx ], y[:train_idx ])
test_loader = torch.utils.data.TensorDataset(features[train_idx :], y[train_idx :])

In [27]:
idx = 1000
batch_size = 64
train_loader = torch.utils.data.TensorDataset(features[:idx], y[:idx ])
test_loader = torch.utils.data.TensorDataset(features[:idx ], y[:idx ])

train_loader = torch.utils.data.DataLoader(train_loader, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_loader, batch_size=batch_size, shuffle=True)

In [28]:
next(iter(train_loader))[1].size()

torch.Size([64, 30])

In [29]:
from networks import *

model  = BiConvGRU_Attention(h=128,conv_feat=100 ,  glove =glove, num_out = num_tags, bidirectional = True)

print(model(Variable(torch.ones(64,100)).long()).size())


torch.Size([64, 30])


In [30]:

params = model.params


opt = optim.Adam(params, lr=0.001)

bce = torch.nn.BCELoss()

def train(train_loader):

    model.train()

    start = time.time()
    avg_loss = 0
    i = 1
    for data, target in train_loader:

        data, target = Variable(data), Variable(target)

        if gpu:

            data = data.cuda()                               
            target = target.cuda()

        opt.zero_grad()

        y_hat = model.forward(data)

        loss = bce(y_hat, target)

        loss.backward()


        opt.step()

        avg_loss += loss
        i += 1

        if i % 700 == 0:
            print("averge loss: ", (avg_loss / i).data[0], " time elapsed:", time.time() - start)
            #print( "CUDA memory: "  ,sbcuda.cuda_ndarray.cuda_ndarray.mem_info()[0]/1024./1024/1024 )

            #for p in params:
            #    print(list(p.size()), (p.data **2 ).mean())# p.data.norm())





def test(test_loader):

    model.eval()

    avg_loss = 0
    
    y_hat_all = np.zeros(y[train_idx:].numpy().shape)
    y_target = np.zeros(y[train_idx:].numpy().shape)

    i = 0

    for data, target in test_loader:

        data, target = Variable(data), Variable(target)

        if gpu:
            target = target.cuda() 
            data = data.cuda()

        y_hat = model.forward(data)

        loss = bce(y_hat, target)

        y_hat = y_hat.cpu().data.numpy()

        y_hat_all[i*data.size()[0]:(i+1)*data.size()[0]] = y_hat
        y_target[i*data.size()[0]:(i+1)*data.size()[0]] = target.data.numpy()

        i+=1

        avg_loss += loss


    #precision, recall, thresholds = precision_recall_curve(y[train_idx :].numpy().flatten(), y_hat_all.flatten())
    precision, recall, thresholds = precision_recall_curve(y_target.flatten(), y_hat_all.flatten())

    
    f_score = 2* precision * recall / (precision + recall)
    i_max = np.nanargmax(f_score)
    f_max = f_score[i_max]
    max_thresh = thresholds[i_max]

    #hamming = hamming_loss(y[train_idx :].numpy(),y_hat_all > max_thresh)
    hamming = hamming_loss(y_target,y_hat_all > max_thresh)

    print("averge loss: ", (avg_loss / len(test_loader)).data[0], " average f score: ", f_max, " average hamming loss: ", hamming)
    print('precision: '  , precision[i_max], 'recall: ' , recall[i_max])



for i in range(20):
    
    train(train_loader)
    test(test_loader)


averge loss:  0.38561582565307617  average f score:  0.273702273702  average hamming loss:  0.006010827121050763
precision:  0.222765363128 recall:  0.354838709677
averge loss:  0.3657945990562439  average f score:  0.275384915221  average hamming loss:  0.00660010649627263
precision:  0.212289663462 recall:  0.391846921797
averge loss:  0.3625631034374237  average f score:  0.284397234192  average hamming loss:  0.006338303159389422
precision:  0.223078132369 recall:  0.392205638474
averge loss:  0.3630577325820923  average f score:  0.283543784006  average hamming loss:  0.006194533191338303
precision:  0.224735557364 recall:  0.3840378198
averge loss:  0.35927093029022217  average f score:  0.290081700782  average hamming loss:  0.007172523961661342
precision:  0.212730318258 recall:  0.455825510768
averge loss:  0.36095917224884033  average f score:  0.295374581653  average hamming loss:  0.007287894923677671
precision:  0.215017491254 recall:  0.471636064675
averge loss:  0.357313

In [31]:
a = torch.ones(64,100,256)

In [32]:
a * torch.ones(256,256)

RuntimeError: inconsistent tensor size at /Users/soumith/anaconda/conda-bld/pytorch-0.1.6_1485018057980/work/torch/lib/TH/generic/THTensorMath.c:623

In [34]:
a = torch.ones(64,100,256)

In [35]:
torch.bmm(a,  torch.ones(64,256,1) ).size()

torch.Size([64, 100, 1])

In [42]:
torch.ones(1,256,1).repeat(4,1,1).size()

torch.Size([4, 256, 1])

In [46]:
a = torch.ones(64,256,100)
torch.bmm( torch.ones(1,1,256).repeat(64,1,1) , a ).size()

torch.Size([64, 1, 100])

In [49]:
torch.ones(1,1,256).expand(64,1,256).size()

torch.Size([64, 1, 256])

In [54]:
from pytube import YouTube
from pprint import pprint
from pytube.exceptions import MultipleObjectsReturned

def on_progress(recieved,size, start):
    
    print('here')

game = 'https://www.youtube.com/watch?v=NDH1bGnNMjw'

yt = YouTube(game)

video = yt.filter('mp4')[0]

video.download('.',on_progress=on_progress )

print('here')

here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
